In [13]:
import pandas as pd

In [14]:
#Global Variables

# File names

## Import
csv_file = 'Video_Games.csv'

## Export
excel_file_name = 'ficheiro.xlsx'
tab1_name = 'tab1'

# Pandas configuration
engine_name = 'xlsxwriter'
encoding_name = 'utf8'


In [15]:
def extract_base():
    return pd.read_csv(csv_file,sep='\t',encoding=encoding_name)
        

In [16]:
def export_xls(in_file):
    writer = pd.ExcelWriter(excel_file_name, engine=engine_name)
    in_file.to_excel(writer, sheet_name=tab1_name,index=False)
    writer.close()

In [17]:
file = extract_base()
export_xls(file)
